In [2]:

# install.packages('dplyr')
# install.packages('jsonlite')
# install.packages('lubridate')
# install.packages('ggplot2')
# install.packages('quantmod')

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)

Installing package into 'C:/Users/samue/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



In [13]:
library(dplyr)
library(jsonlite)
library(lubridate)
library(ggplot2)
library(quantmod)
library(lifecycle)
library(tidyselect)



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



######################### Warning from 'xts' package ##########################
#                                                                             #
# The dplyr lag() function breaks how base R's lag() function is supposed to  #
# work, which breaks lag(my_xts). Calls to lag(my_xts) that you type or       #
# source() into this session won't work correctly.                            #
#                                                                             #
# Use stats::lag() to make sure you're not using dplyr::lag(), or you can add #
# conflictRules('dplyr', exclude = 'lag') to your .Rprofile to stop           #
# dplyr from breaking base R's lag() function.          

In [14]:
# Load necessary libraries
library(ggplot2)
library(dplyr)
library(jsonlite)

# Read the CSV file into a dataframe
WIN_PRO_df <- read.csv("win_data_pro.csv")

# Remove rows where any entry in the trades column contains 0
WIN_PRO_df <- WIN_PRO_df %>% filter(trades != "[]")

# Convert timestamp to POSIXct
WIN_PRO_df <- WIN_PRO_df %>%
  mutate(timestamp = as.POSIXct(timestamp_unix / 1000, origin = "1970-01-01", tz = "GMT"))

# Convert the string representations to actual lists
WIN_PRO_df$trades <- lapply(WIN_PRO_df$trades, function(x) fromJSON(as.character(x)))

# Directly assign last_traded_price to price and set size to NA
WIN_PRO_df <- WIN_PRO_df %>%
  mutate(
    size = NA,
    price = last_traded_price
  )

# Remove trades column
WIN_PRO_df <- WIN_PRO_df %>% select(-trades)

# Read the horse name data into a dataframe
horse_names_df <- data.frame(
  horse_id = c(8421889, 5105924, 5465145, 3415981, 7560122, 7401388),
  horse_name = c("Romsdal", "Brown Panther", "Havana Beat", "Times Up", "Snow Sky", "Island Remede")
)

# Join the horse name data with the main dataframe
WIN_PRO_df <- WIN_PRO_df %>%
  left_join(horse_names_df, by = c("horse_id" = "horse_id"))

# Important instances to split df
PP_t <- "2015-05-15 16:10:00 GMT"
IP_t <- "2015-05-15 16:16:10 GMT"

# Split the data frame into three parts based on the given times
WIN_PRO_df_Morning <- WIN_PRO_df %>% filter(timestamp < PP_t)
WIN_PRO_df_PP <- WIN_PRO_df %>% filter(timestamp >= PP_t & timestamp < IP_t)
WIN_PRO_df_IP <- WIN_PRO_df %>% filter(timestamp >= IP_t)

# Loop through each horse and create a plot and save dataframe
for (horse in unique(WIN_PRO_df$horse_name)) {
  horse_df <- WIN_PRO_df_PP %>% filter(horse_name == horse)
  
  # Calculate the size column for each horse within the loop
  horse_df <- horse_df %>%
    arrange(timestamp) %>%
    mutate(size = traded_volume - lag(traded_volume, default = first(traded_volume)),
           size = ifelse(size < 0, 0, size))
  
  # Print a sample of the horse-specific dataframe to check the size column
  print(head(horse_df))
  
  # Save the horse-specific dataframe to a variable named after the horse
  assign(paste0(gsub(" ", "_", horse), "_WIN_df"), horse_df)
  
  # Create the plot with only price
  p <- ggplot(data = horse_df, aes(x = timestamp, y = price)) +
    geom_line() +
    geom_point() +
    labs(
      title = paste("WIN market Price Over Time for", horse),
      x = "Time",
      y = "Price"
    ) +
    theme_minimal() +
    theme(
      axis.text.x = element_text(angle = 45, hjust = 1),
      legend.position = "bottom"
    ) +
    scale_x_datetime(date_breaks = "1 min", date_labels = "%H:%M")
  
  # Print the plot
  print(p)
}
